In [44]:
!pip install -q langchain-openai langchain-core requests

In [45]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [47]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [48]:
multiply.name

'multiply'

In [49]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [50]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [51]:
# tool binding

In [52]:
llm = ChatGroq(model='moonshotai/kimi-k2-instruct')

In [53]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 27, 'total_tokens': 38, 'completion_time': 0.022032507, 'completion_tokens_details': None, 'prompt_time': 0.009679599, 'prompt_tokens_details': None, 'queue_time': 0.197163542, 'total_time': 0.031712106}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bf504-adce-7421-8b36-045cae9018f7-0', usage_metadata={'input_tokens': 27, 'output_tokens': 11, 'total_tokens': 38})

In [54]:
llm_with_tools = llm.bind_tools([multiply])

In [55]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="Hi! I'm doing well, thanks for asking! How about you?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 110, 'total_tokens': 125, 'completion_time': 0.037627524, 'completion_tokens_details': None, 'prompt_time': 0.014979089, 'prompt_tokens_details': None, 'queue_time': 0.197127258, 'total_time': 0.052606613}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bf504-c4b9-7a92-82d8-d6c0abdbaa4d-0', usage_metadata={'input_tokens': 110, 'output_tokens': 15, 'total_tokens': 125})

In [56]:
query = HumanMessage('can you multiply 3 with 1000')

In [57]:
messages = [query]

In [58]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [59]:
result = llm_with_tools.invoke(messages)

In [60]:
messages.append(result)

In [61]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'll multiply 3 with 1000 for you.", additional_kwargs={'tool_calls': [{'id': 'functions.multiply:0', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 115, 'total_tokens': 149, 'completion_time': 0.087136365, 'completion_tokens_details': None, 'prompt_time': 0.015452751, 'prompt_tokens_details': None, 'queue_time': 0.197159559, 'total_time': 0.102589116}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bf504-c6a5-7fe2-8d66-603e7506bc19-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'functions.multiply:0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 115, 'output_tok

In [62]:
tool_result = multiply.invoke(result.tool_calls[0])

In [63]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='functions.multiply:0')

In [64]:
messages.append(tool_result)

In [65]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'll multiply 3 with 1000 for you.", additional_kwargs={'tool_calls': [{'id': 'functions.multiply:0', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 115, 'total_tokens': 149, 'completion_time': 0.087136365, 'completion_tokens_details': None, 'prompt_time': 0.015452751, 'prompt_tokens_details': None, 'queue_time': 0.197159559, 'total_time': 0.102589116}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bf504-c6a5-7fe2-8d66-603e7506bc19-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'functions.multiply:0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 115, 'output_tok

In [66]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 with 1000 is **3000**.'

In [67]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [68]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [69]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1769299202,
 'time_last_update_utc': 'Sun, 25 Jan 2026 00:00:02 +0000',
 'time_next_update_unix': 1769385602,
 'time_next_update_utc': 'Mon, 26 Jan 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 91.8485}

In [70]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [71]:
# tool binding
llm = ChatGroq(model='moonshotai/kimi-k2-instruct')

In [72]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [73]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [74]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [75]:
ai_message = llm_with_tools.invoke(messages)

In [76]:
messages.append(ai_message)

In [77]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'functions.get_conversion_factor:0',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'functions.convert:1',
  'type': 'tool_call'}]

In [78]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [79]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'll help you get the conversion factor between INR and USD and then convert 10 INR to USD.", additional_kwargs={'tool_calls': [{'id': 'functions.get_conversion_factor:0', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'functions.convert:1', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 212, 'total_tokens': 274, 'completion_time': 0.18420283, 'completion_tokens_details': None, 'prompt_time': 0.023648472, 'prompt_tokens_details': None, 'queue_time': 0.192771569, 'total_time': 0.207851302}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_5fe129dff6', 'service_ti

In [80]:
llm_with_tools.invoke(messages).content

'Based on the current exchange rate:\n\n**Conversion factor:** 1 INR = 0.01089 USD\n\n**Conversion result:** 10 INR = 0.1089 USD\n\nSo, 10 Indian Rupees converts to approximately 0.11 US Dollars (or 10.89 cents).'

In [43]:
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain import hub

# Step 5: Initialize the Agent ---
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

agent = create_structured_chat_agent(llm, [get_conversion_factor, convert], prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[get_conversion_factor, convert],
    verbose=True,
    handle_parsing_errors=True
)

ImportError: cannot import name 'create_structured_chat_agent' from 'langchain.agents' (/home/dark/miniconda3/envs/genai_env/lib/python3.12/site-packages/langchain/agents/__init__.py)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here and ready to help! How can I assist you today?

> Finished chain.
